<a href="https://colab.research.google.com/github/AreYouRuben/DS_Team24/blob/main/Machine-Learning-Santander/is_Podium/Bayes_v_Forest_DAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df_pit = pd.read_csv('https://raw.githubusercontent.com/AreYouRuben/DS_Team24/main/Analisis_Datos_Python_Santander/f1_dataset/pit_stops.csv')
df_pit = df_pit[['raceId','driverId','stop','lap']]
df_pit

,raceId,driverId,stop,lap
0,841,153,1,1
1,841,30,1,1
2,841,17,1,11
3,841,4,1,12
4,841,13,1,13
...,...,...,...,...
8181,1054,817,3,41
8182,1054,853,4,43
8183,1054,815,3,51
8184,1054,822,4,63


In [3]:
df_pit = df_pit.groupby(['raceId','driverId'])['stop','lap'].max()
df_pit = df_pit.reset_index()
df_pit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,raceId,driverId,stop,lap
0,841,1,2,36
1,841,2,2,30
2,841,3,1,16
3,841,4,3,42
4,841,5,1,17
...,...,...,...,...
3946,1054,847,3,28
3947,1054,849,3,24
3948,1054,852,3,21
3949,1054,853,4,43


In [4]:
df_result = pd.read_csv('https://raw.githubusercontent.com/AreYouRuben/DS_Team24/main/Analisis_Datos_Python_Santander/f1_dataset/results.csv')
df_result = df_result[['raceId','driverId','constructorId','grid','laps','position']]
df_result.head(5)

,raceId,driverId,constructorId,grid,laps,position
0,18,1,1,1,58,1
1,18,2,2,5,58,2
2,18,3,3,7,58,3
3,18,4,4,11,58,4
4,18,5,1,3,58,5


In [5]:
df_races = pd.read_csv('https://raw.githubusercontent.com/AreYouRuben/DS_Team24/main/Analisis_Datos_Python_Santander/f1_dataset/races.csv')
df_races = df_races[df_races['year'] > 2000]
df_races = df_races[['raceId','circuitId']]
df_races

,raceId,circuitId
0,1,1
1,2,2
2,3,17
3,4,3
4,5,4
...,...,...
1053,1069,69
1054,1070,32
1055,1071,18
1056,1072,77


In [6]:
df_merged = df_pit.merge(df_result,how='inner',left_on=['raceId','driverId'],right_on=['raceId','driverId'])
df_merged = df_merged.merge(df_races,how='inner',left_on='raceId',right_on='raceId')
df_merged

,raceId,driverId,stop,lap,constructorId,grid,laps,position,circuitId
0,841,1,2,36,1,2,58,2,1
1,841,2,2,30,4,18,57,12,1
2,841,3,1,16,131,7,22,\N,1
3,841,4,3,42,6,5,58,4,1
4,841,5,1,17,205,19,19,\N,1
...,...,...,...,...,...,...,...,...,...
3946,1054,847,3,28,3,11,65,16,75
3947,1054,849,3,24,3,18,64,18,75
3948,1054,852,3,21,213,14,65,15,75
3949,1054,853,4,43,210,20,64,19,75


In [7]:
df_merged.replace(r'\N', np.nan, inplace=True)
df_merged.isna().sum(axis=0)

raceId             0
driverId           0
stop               0
lap                0
constructorId      0
grid               0
laps               0
position         430
circuitId          0
dtype: int64

In [8]:
df_merged = df_merged.dropna(subset=['position'])
df_merged.isna().sum(axis=0)

raceId           0
driverId         0
stop             0
lap              0
constructorId    0
grid             0
laps             0
position         0
circuitId        0
dtype: int64

In [9]:
df_merged = df_merged.reset_index(drop=True)
df = df_merged
df

,raceId,driverId,stop,lap,constructorId,grid,laps,position,circuitId
0,841,1,2,36,1,2,58,2,1
1,841,2,2,30,4,18,57,12,1
2,841,4,3,42,6,5,58,4,1
3,841,13,3,48,6,8,58,7,1
4,841,15,2,37,205,20,56,13,1
...,...,...,...,...,...,...,...,...,...
3516,1054,847,3,28,3,11,65,16,75
3517,1054,849,3,24,3,18,64,18,75
3518,1054,852,3,21,213,14,65,15,75
3519,1054,853,4,43,210,20,64,19,75


In [10]:
df['position'] =  pd.to_numeric(df['position'])
df.dtypes

raceId           int64
driverId         int64
stop             int64
lap              int64
constructorId    int64
grid             int64
laps             int64
position         int64
circuitId        int64
dtype: object

In [11]:
df['podium'] = np.where(df['position'] < 4, 1, 0)
df

,raceId,driverId,stop,lap,constructorId,grid,laps,position,circuitId,podium
0,841,1,2,36,1,2,58,2,1,1
1,841,2,2,30,4,18,57,12,1,0
2,841,4,3,42,6,5,58,4,1,0
3,841,13,3,48,6,8,58,7,1,0
4,841,15,2,37,205,20,56,13,1,0
...,...,...,...,...,...,...,...,...,...,...
3516,1054,847,3,28,3,11,65,16,75,0
3517,1054,849,3,24,3,18,64,18,75,0
3518,1054,852,3,21,213,14,65,15,75,0
3519,1054,853,4,43,210,20,64,19,75,0


In [12]:
df.to_csv('is_Podium.csv', index=False)